In [2]:
library(anndata)
library(Signac)
library(Seurat)
library(anndata)
library(ggplot2)
library(tidyverse)
library(Signac)
library(Seurat)
#library(EnsDb.Mmusculus.v102)
library(BSgenome.Mmusculus.UCSC.mm10)
#library(EnsDb.Mmusculus.v102)
#library(pastecs)
library(stringr)

In [3]:
R.version

               _                           
platform       x86_64-apple-darwin13.4.0   
arch           x86_64                      
os             darwin13.4.0                
system         x86_64, darwin13.4.0        
status                                     
major          4                           
minor          1.3                         
year           2022                        
month          03                          
day            10                          
svn rev        81868                       
language       R                           
version.string R version 4.1.3 (2022-03-10)
nickname       One Push-Up                 

In [4]:
#devtools::install_github("GreenleafLab/chromVARmotifs")

In [5]:
library("chromVARmotifs")

In [6]:
data("mouse_pwms_v2")

In [7]:
sout <- sapply(strsplit(names(mouse_pwms_v2), split = "_"), function(s) c(s[3]))
mouse_pwms_v2 <- mouse_pwms_v2[match(unique(sout), sout)]

In [8]:
seurat <- readRDS("/Users/dominik.klein/data/seurat_filtered.rds")

In [9]:
BSgenome.Mmusculus.UCSC.mm10.renamed <- renameSeqlevels(BSgenome.Mmusculus.UCSC.mm10, value = str_replace(str_replace(seqnames(BSgenome.Mmusculus.UCSC.mm10), pattern = "chr", replacement = ""), pattern = "M", replacement = "MT"))

In [10]:
# add motif information
seurat <- AddMotifs(
  object = seurat,
  genome = BSgenome.Mmusculus.UCSC.mm10.renamed,
  pfm = mouse_pwms_v2
)

Building motif matrix

Finding motif positions

Creating Motif object



In [11]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("chromVAR")

In [12]:
library("chromVAR")

In [13]:
seurat <- RunChromVAR(
  object = seurat,
  genome = BSgenome.Mmusculus.UCSC.mm10.renamed
)



Computing GC bias per region

Selecting background regions

Computing deviations from background

Constructing chromVAR assay

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [14]:
DefaultAssay(seurat) <- "chromvar"

In [15]:
seurat

An object of class Seurat 
476213 features across 16918 samples within 4 assays 
Active assay: chromvar (884 features, 0 variable features)
 3 other assays present: RNA, ATAC, ATAC_merged
 1 dimensional reduction calculated: lsi

In [16]:
write.csv(seurat[["chromvar"]]@data, "/Users/dominik.klein/data/chromvar_annotations_reduced.csv")

In [18]:
saveRDS(seurat, "/Users/dominik.klein/data/seurat_with_chromVAR_reduced.rds")

In [17]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur/Monterey 10.16

Matrix products: default
BLAS/LAPACK: /Users/dominik.klein/opt/anaconda3/envs/mm_best_muon_full/lib/libopenblasp-r0.3.20.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] chromVAR_1.16.0                    chromVARmotifs_0.2.0              
 [3] BSgenome.Mmusculus.UCSC.mm10_1.4.3 BSgenome_1.62.0                   
 [5] rtracklayer_1.54.0                 Biostrings_2.62.0                 
 [7] XVector_0.34.0                     GenomicRanges_1.46.1              
 [9] GenomeInfoDb_1.30.1                IRanges_2.28.0                    
[11] S4Vectors_0.32.4                   BiocGenerics_0.40.0               
[13] forcats_0.5.1                      stringr_1.4.0                     
[15] dplyr_1.0.9                        purrr_0.3.4          